In [ ]:
# !pip install -U langchain  nest_asyncio httpx redis[hiredis] redisearch > 2.4 bitsandbytes

In [27]:
# !pip install -U llama-index

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
# from langchain import llms
# dir(llms)

In [1]:
import os
import json
import uuid
import torch
import requests
from typing import Any, List, Mapping, Optional
from dotenv import load_dotenv
import langchain 
from threading import Thread
from tqdm import tqdm
from peft import PeftModel
from IPython.display import Markdown, display

from pydantic import BaseModel, Field
from transformers import pipeline, TextStreamer, TextIteratorStreamer, LlamaTokenizer, LlamaForCausalLM

from llama_index.indices.composability import ComposableGraph
from llama_index.prompts.prompts import QuestionAnswerPrompt, RefinePrompt
from llama_index.langchain_helpers.memory_wrapper import GPTIndexChatMemory
from llama_index.langchain_helpers.agents import LlamaToolkit, create_llama_chat_agent, IndexToolConfig, LlamaIndexTool
from llama_index import download_loader, SummaryPrompt, LLMPredictor, GPTListIndex, GPTVectorStoreIndex, PromptHelper, load_index_from_storage, StorageContext, ServiceContext, LangchainEmbedding, SimpleDirectoryReader
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index.node_parser import SimpleNodeParser, NodeParser
from llama_index.vector_stores import ChromaVectorStore


from langchain.tools import BaseTool, StructuredTool, tool
from langchain.prompts import MessagesPlaceholder
from langchain.agents import Tool, AgentOutputParser
from langchain import OpenAI, LLMChain
from langchain.llms.base import LLM
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate
from langchain.callbacks import tracing_enabled
from langchain.agents import initialize_agent, LLMSingleActionAgent, StructuredChatAgent, ConversationalChatAgent, ConversationalAgent, AgentExecutor, ZeroShotAgent, AgentType
from langchain.embeddings import HuggingFaceEmbeddings, HuggingFaceInstructEmbeddings
from langchain.chains.conversation.memory import ConversationBufferMemory, ConversationStringBufferMemory, ConversationBufferWindowMemory
from langchain.memory.chat_memory import ChatMessageHistory
from langchain.memory import ConversationKGMemory
from langchain.memory.chat_message_histories import RedisChatMessageHistory
from langchain.cache import RedisSemanticCache

load_dotenv()

/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/paths.py:98: UserWarning: /opt/conda did not contain libcudart.so as expected! Searching further paths...
  warn(


CUDA SETUP: CUDA path found: /usr/local/cuda/targets/x86_64-linux/lib/libcudart.so
CUDA_SETUP: Detected CUDA version 118
CUDA_SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


True

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
# define prompt helper
# set maximum input size
max_input_size = 4096
# set number of output tokens
num_output = 2048
# set maximum chunk overlap
max_chunk_overlap = 20
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

### Custom Model

In [4]:
# model_name = "jagadeesh/vicuna-13b"
# tokenizer = LlamaTokenizer.from_pretrained(model_name,  use_fast=False)
# streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, Timeout=5)
# model = LlamaForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True, device_map="auto", load_in_8bit= False, use_auth_token=True, torch_dtype=torch.float16 )

In [5]:
# class CustomLLM(LLM):
#     model_name = "jagadeesh/vicuna-13b"
#     def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
#         inputs = tokenizer([prompt], return_tensors="pt")

#         # Run the generation in a separate thread, so that we can fetch the generated text in a non-blocking way.
#         generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=num_output)
#         thread = Thread(target=model.generate, kwargs=generation_kwargs)
#         thread.start()
        
#         return ""

#     @property
#     def _identifying_params(self) -> Mapping[str, Any]:
#         return {"name_of_model": self.model_name}

#     @property
#     def _llm_type(self) -> str:
#         return "custom"

# llm_predictor = LLMPredictor(llm=CustomLLM())

### HF Pipeline vicuna 13b

In [6]:
model_id = "jagadeesh/vicuna-13b"
tokenizer = LlamaTokenizer.from_pretrained(model_id, use_fast=False)
streamer = TextStreamer(tokenizer, skip_prompt=True, Timeout=5)
model = LlamaForCausalLM.from_pretrained(model_id, load_in_8bit=False, low_cpu_mem_usage=True, device_map="auto", max_memory={0:"18GB",1:"18GB",2:"18GB",3:"18GB","cpu":"10GB"}, torch_dtype=torch.float16)
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, streamer=streamer, max_new_tokens=num_output, device_map="auto"
)
hf_pipeline = HuggingFacePipeline(pipeline=pipe)
llm_predictor = LLMPredictor(llm=hf_pipeline)

In [7]:
# api = "hf_"

# tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")

# import huggingface_hub as hf_hub

# hf_hub.login(token=api)

# ## loading llama base model and configuring it with adapter

# base_model_name = 'decapoda-research/llama-7b-hf'

# base_model = LlamaForCausalLM.from_pretrained(
#             base_model_name,
#             torch_dtype=torch.float16,
#             device_map="auto",
#         )

# model = PeftModel.from_pretrained(
#             base_model,
# #             'shrinath-suresh/alpaca-lora-7b-delta',
#             'shrinath-suresh/alpaca-lora-all-7b-delta',
#             torch_dtype=torch.float16,
#             load_in_8bit=True
#         )
# class CustomLLM(LLM):
#     model_name = "shrinath-suresh/alpaca-lora-all-7b-delta"
#     def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
#         inputs = tokenizer([prompt], return_tensors="pt")

#         # Run the generation in a separate thread, so that we can fetch the generated text in a non-blocking way.
#         response = model.generate(**inputs, max_new_tokens=num_output)
#         return response

#     @property
#     def _identifying_params(self) -> Mapping[str, Any]:
#         return {"name_of_model": self.model_name}

#     @property
#     def _llm_type(self) -> str:
#         return "custom"

# llm_predictor = LLMPredictor(llm=CustomLLM())

In [8]:
# embed_model = LangchainEmbedding(HuggingFaceEmbeddings())
HF_embed_model = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
embed_model = LangchainEmbedding(HF_embed_model)

load INSTRUCTOR_Transformer
max_seq_length  512


In [9]:
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper, embed_model=embed_model)

## Load data from disk

In [527]:
import llama_index
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage.index_store import SimpleIndexStore
from llama_index.vector_stores import SimpleVectorStore
storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="./pytorch_docs_1024_code_parser"),
    vector_store=SimpleVectorStore.from_persist_dir(persist_dir="./pytorch_docs_1024_code_parser"),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="./pytorch_docs_1024_code_parser"),
)
index = load_index_from_storage(storage_context, service_context=service_context)

In [528]:
retriver_engine = index.as_retriever(service_context=service_context)
nodes = retriver_engine.retrieve("How to create pytorch nn module")

### Dataset Tool

In [ ]:
API_TOKEN="hf_"

In [460]:
from transformers.tools import Tool
from huggingface_hub import list_models, ModelFilter
import huggingface_hub as hf_hub
import re

class get_query(BaseModel):
    query: str = Field(default=None, description="The name of the model and task for which you want the datasets.")

@tool
def get_datasets(query: str) -> str:
    """Fetches dataset for a given model and task"""
    print("Log::::::::::::::::::::::::::::", query, type(query))
    pattern = r'model=(.*?)\s*(?:and\s*)?task=(.*?)$'
    match = re.search(pattern, query)
    model_name = match.group(1)
    task = match.group(2)
    print("Log::::::::::::::::::::::::::::", model_name, task)

    filter = ModelFilter(library="pytorch", model_name=model_name, task=task)

    models = list_models(filter=filter, token=API_TOKEN)    
    if not models:
        return f"{model_name} Not Found!"
    model = [model for model in models if model.id == model_name]
    datasets = [string.split(":")[1] for string in model[0].tags if "dataset:" in string]
    if not datasets:
        return f"No Datasets available for {model_name}"
    print("Log::::::::::::::::::::::::::::",",".join(datasets))
    return ",".join(datasets)

In [461]:
search_tools = [
    StructuredTool.from_function(
        func=get_datasets,
        name="Get Model Datasets",
        args_schema=get_query,
        return_direct=True,
        description="A search engine. Useful for when you need to answer questions about fetching datasets. For example the input should be like model=model and task=task. Do not use this tool with the same input/query",
    ),
]
index_tools = [
    LlamaIndexTool.from_tool_config(
        IndexToolConfig(
            name = "PyTorch Index",
            query_engine=index.as_query_engine(similarity_top_k=3, response_mode="compact", service_context=service_context),
            description=f"useful for answering questions related to pytorch. Do not use this tool for fetching dataset. Do not use this tool with the same input/query",
            return_direct=True
        )
    ),
]

ALL_TOOLS = search_tools+index_tools

In [462]:
ALL_TOOLS

[StructuredTool(name='Get Model Datasets', description="Get Model Datasets(tool_input: 'str', callbacks: 'Callbacks' = None) -> 'str' - A search engine. Useful for when you need to answer questions about fetching datasets. For example the input should be like model=model and task=task. Do not use this tool with the same input/query", args_schema=<class '__main__.get_query'>, return_direct=True, verbose=False, callbacks=None, callback_manager=None, func=StructuredTool(name='get_datasets', description='get_datasets(query: str) -> str - Fetches dataset for a given model and task', args_schema=<class 'pydantic.main.get_datasetsSchemaSchema'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, func=<function get_datasets at 0x7f01c8028f70>, coroutine=None), coroutine=None),
 LlamaIndexTool(name='PyTorch Index', description='useful for answering questions related to pytorch. Do not use this tool for fetching dataset. Do not use this tool with the same input/query', ar

### Tool Retriever

We will use a vectorstore to create embeddings for each tool description. Then, for an incoming query we can create embeddings for that query and do a similarity search for relevant tools.


In [463]:
from langchain.vectorstores import FAISS
from langchain.schema import Document

docs = [Document(page_content=t.description, metadata={"index": i}) for i, t in enumerate(ALL_TOOLS)]

vector_store = FAISS.from_documents(docs, HF_embed_model)

retriever = vector_store.as_retriever()

def get_tools(query):
    docs = retriever.get_relevant_documents(query)
    return [ALL_TOOLS[d.metadata["index"]] for d in docs]

In [464]:
get_tools("give me the supported datasets for the {\"model\":\"bert-base-uncased\", \"task\":\"fill-mask\"}")

[StructuredTool(name='Get Model Datasets', description="Get Model Datasets(tool_input: 'str', callbacks: 'Callbacks' = None) -> 'str' - A search engine. Useful for when you need to answer questions about fetching datasets. For example the input should be like model=model and task=task. Do not use this tool with the same input/query", args_schema=<class '__main__.get_query'>, return_direct=True, verbose=False, callbacks=None, callback_manager=None, func=StructuredTool(name='get_datasets', description='get_datasets(query: str) -> str - Fetches dataset for a given model and task', args_schema=<class 'pydantic.main.get_datasetsSchemaSchema'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, func=<function get_datasets at 0x7f01c8028f70>, coroutine=None), coroutine=None),
 LlamaIndexTool(name='PyTorch Index', description='useful for answering questions related to pytorch. Do not use this tool for fetching dataset. Do not use this tool with the same input/query', ar

### Enable aching

In [467]:
# langchain.llm_cache = RedisSemanticCache(
#     redis_url="redis://localhost:6379",
#     embedding=HuggingFaceEmbeddings()
# )
langchain.llm_cache = None

### Output Parser

In [468]:
import re
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
output_parser = CustomOutputParser()

# Prompt

In [471]:
from langchain.prompts import StringPromptTemplate
from typing import Callable
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish

# Set up the base template
template_with_history = """Act like you are an expert PyTorch Engineer and provide answers to these questions from the developer community. 
If you don't know the answer say "I am not sure about this, can you post the question on pytorch-discuss channel", don't make up an answer if you don't know. 

You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat 1 times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools_getter: Callable
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        ############## NEW ######################
        tools = self.tools_getter(kwargs["input"])
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in tools])
        return self.template.format(**kwargs)

prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools_getter=get_tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)


In [472]:
prompt_with_history.template

'Act like you are an expert PyTorch Engineer and provide answers to these questions from the developer community. \nIf you don\'t know the answer say "I am not sure about this, can you post the question on pytorch-discuss channel", don\'t make up an answer if you don\'t know. \n\nYou have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat 1 times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s\n\nPrevious conversation history:\n{history}\n\nNew question: {input}\n{agent_scratchpad}'

### Callback

In [474]:
# from langchain.callbacks.base import BaseCallbackHandler

# class MyCustomHandler(BaseCallbackHandler):
#     def on_llm_new_token(self, token: str, **kwargs) -> None:
#         print(f"My custom handler, token: {token}")

### Create llm chain and agent

In [503]:
tools = get_tools("")
tool_names = [tool.name for tool in tools]
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# llm = OpenAI(streaming=True, temperature=0, callbacks=[StreamingStdOutCallbackHandler()], openai_api_key="sk-")
llm = hf_pipeline

llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)
# llm_chain = LLMChain(llm=CustomLLM(), prompt=prompt)

agent = LLMSingleActionAgent(llm_chain=llm_chain, output_parser=output_parser, stop=["\nObservation: "], allowed_tools=tool_names)

### Create agent chain and session history 

In [504]:
session_id = uuid.uuid4()

message_history = RedisChatMessageHistory(str(session_id), 'redis://localhost:6379/0', ttl=600)
memory = ConversationBufferWindowMemory(k=2, memory_key="history", return_messages=True, chat_memory=message_history)

agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, streaming=True, tools=tools, verbose=False, memory=memory)

In [505]:
# memory.clear()

### Run query

In [506]:
response = agent_chain.run(input="give me the datasets for the model=bert-base-uncased and task=fill-mask")
print("=========================================")
print(response)
print("=========================================")


Thought: I need to find the datasets for the given model and task
Action: Get Model Datasets
Action Input: model=bert-base-uncased and task=fill-maskLog:::::::::::::::::::::::::::: model=bert-base-uncased and task=fill-mask <class 'str'>
Log:::::::::::::::::::::::::::: bert-base-uncased fill-mask
Log:::::::::::::::::::::::::::: bookcorpus,wikipedia
bookcorpus,wikipedia


In [455]:
response = agent_chain.run(input="What's the difference between reshape and view in pytorch?")
#  and task 'image-classification'
print("=========================================")
print(response)
print("=========================================")

Thought: I need to understand the difference between reshape and view
Action: PyTorch Index
Action Input: reshape view pytorch
reshape view pytorch
---------------

What is the purpose of the `reshape` function in PyTorch?

a) To create a new tensor with the same data and number of elements as the input tensor, but with a different shape
b) To create a new tensor with a different data and number of elements as the input tensor, but with the same shape
c) To create a new tensor with the same data and number of elements as the input tensor, but with a different data type
d) To create a new tensor with a different data and number of elements as the input tensor, but with a different data type

What 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_30582/2330488204.py:1 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_30582/2330488204.py'                        │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/langchain/chains/base.py:239 in run                      │
│                                                                                                  │
│   236 │   │   │   return self(args[0], callbacks=callbacks)[self.output_keys[0]]                 │
│   237 │   │                                                                                      │
│   238 │   │   if kwargs and not args:                                                            │
│ ❱ 239 │   │   │   return self(kwargs, callbacks=callbacks)[self.output_keys[0]]                  │
│   240 │   │                                                                                      │
│   241 │   │   if not kwargs and not args:                                                        │
│   242 │   │   │   raise ValueError(                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/langchain/chains/base.py:140 in __call__                 │
│                                                                                                  │
│   137 │   │   │   )                                                                              │
│   138 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   139 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 140 │   │   │   raise e                                                                        │
│   141 │   │   run_manager.on_chain_end(outputs)                                                  │
│   142 │   │   return self.prep_outputs(inputs, outputs, return_only_outputs)                     │
│   143                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/langchain/chains/base.py:134 in __call__                 │
│                                                                                                  │
│   131 │   │   )                                                                                  │
│   132 │   │   try:                                                                               │
│   133 │   │   │   outputs = (                                                                    │
│ ❱ 134 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   135 │   │   │   │   if new_arg_supported                                                       │
│   136 │   │   │   │   else self._call(inputs)                                                    │
│   137 │   │   │   )                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/langchain/agents/agent.py:953 in _call                   │
│                                                                                                  │
│    950 │   │   start_time = time.time()                                                          │
│    951 │   │   # We now enter the agent loop (until it returns something).                       │
│    952 │   │   while self._should_continue(iterations, time